# Datensatz 2 (Schrauben)

In diesem Abschnitt soll ein Datensatz aus "echten" Fotos erstellt werden.

Wie viele Trainingsdaten benötigt das Netz? Sind [genug Trainingsdaten](https://towardsdatascience.com/how-do-you-know-you-have-enough-training-data-ad9b1fd679ee) vorhanden?

:::{note}
Es stellt sich die Frage wie detailreich die Schraubenbilder sein müssen um eine gute Erkennung zu ermöglichen?
:::

Um kostengünstig und schnell eine vielzahl an Trainingsbildern von Schrauben zu erzeugen, bieten sich Videoaufnahmen der Schrauben an. Die Videoaufnahmen lassen sich in einzelne Bildern zerlegen und somit ein Datensatz erzeugen.

## Extracting and Saving Video Frames using OpenCV-Python

```
# OpenCV importieren:

import cv2



# path = 'relativer Speicherpfad / Dateiname':

path = 'pozi/pozi'



# Video laden:

cap = cv2.VideoCapture('pozi.MOV')
i = 0


# Prüfen ob ein Video geladen wurde:

if cap.isOpened() == False:
    print('ERROR: Datei nicht gefunden')

    
    
# Die Frames des Videos lesen:    
    
while(cap.isOpened()):
    ret, frame = cap.read()
     
    # sobald keine Frames mehr gelesen werden können (ret==False) wird abgebrochen:
    if ret == False:
        break
     
    # Die Frames speichern
    cv2.imwrite(path+str(i)+'.jpg', frame)
    i += 1
 

cap.release()
cv.destroyAllWindows()
```

### Resize / Crop Image

In [ ]:
```
import cv2

img = cv2.imread('pozi/pozi800.jpg', cv2.IMREAD_UNCHANGED)

print('Original Dimensions : ',img.shape)

scale_percent = 50 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)

#resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

print('Resized Dimensions : ',resized.shape)

cv2.imshow("Resized image", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
```

In [113]:
import glob
import numpy as np
from PIL import Image
filelist = glob.glob("screw_img/**/*")
X_raw = np.array([np.array(Image.open(fname)) for fname in filelist])
print(X_raw.shape)
print(len(filelist))

# Array erweitern - hier nicht nötig
#X_raw = X_raw.reshape(X_raw.shape + (1,))
#print(X_raw.shape)

(10800, 224, 224, 3)
10800


In [114]:
filelist = glob.glob("screw_img/**/*")
X_gray = np.array([np.array(Image.open(fname).convert('L')) for fname in filelist])
print(X_gray.shape)
print(len(filelist))


(10800, 224, 224)
10800


In [ ]:
# Array erweitern 
#X_gray = X_gray.reshape(X_gray.shape + (1,))
print(X_gray.shape)

### Bilderklassen in separatem Ordner

Die folgende Ordnerstruktur liegt vor:

main_directory/  
...category_a/  
......a_image_1.jpg  
......a_image_2.jpg  
...category_b/  
......b_image_1.jpg  
......b_image_2.jpg  

In [66]:
training_data = []

IMG_SIZE=224


def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

100%|█████████████████████████████████████████████████████████████████████████████| 1800/1800 [00:03<00:00, 582.57it/s]

10800


In [102]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))


X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE)

[[[[189]
   [189]
   [191]
   ...
   [191]
   [191]
   [196]]

  [[188]
   [188]
   [189]
   ...
   [192]
   [193]
   [197]]

  [[186]
   [185]
   [185]
   ...
   [194]
   [190]
   [191]]

  ...

  [[171]
   [175]
   [180]
   ...
   [176]
   [174]
   [179]]

  [[164]
   [175]
   [183]
   ...
   [174]
   [177]
   [182]]

  [[168]
   [178]
   [178]
   ...
   [171]
   [177]
   [182]]]]


In [103]:
print(X.shape)
len(y)

(10800, 224, 224)


10800

## Datensatz aufteilen in Test- und Trainingsdaten

In [104]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [105]:
from numpy import save, load
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('Dataset_224x224/X_train.npy', X_train)
save('Dataset_224x224/y_train.npy', y_train)
save('Dataset_224x224/X_test.npy', X_test)
save('Dataset_224x224/y_test.npy', y_test)

## Datensatz mit Keras erzeugen und erweitern

Mit dem ImageDataGenerator aus dem Keras Paket *Image data preprocessing*, kann ein Datensatz erstellt werden.
Der Image Data Generator bietet eine Reihe nützlicher Funktionen um einen Datensatz aus Bildern zu erstellen und zu erweitern.

In [ ]:
import tensorflow
from tensorflow import keras

In [ ]:
data = []
labels = []

In [107]:
# Create a dataset.
dataset = keras.preprocessing.image_dataset_from_directory(
  'screw_img', batch_size=100, image_size=(224, 224))

data = []
labels = []




# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
    print(data.shape)  # (64, 200, 200, 3)
    print(data.dtype)  # float32
    print(labels.shape)  # (64,)
    print(labels.dtype)  # int32
    break
    
   

Found 10804 files belonging to 6 classes.
(100, 224, 224, 3)
<dtype: 'float32'>
(100,)
<dtype: 'int32'>


In [108]:
X_ = []
y_ = []

for features,label in dataset:
    X_.append(features)
    y_.append(label)

In [110]:
print(X_.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(X.shape)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
class_names = X.class_names
print(len(class_names))

## Generator ausprobieren und konfigurieren:

In [ ]:
#Generator config:
gen = ImageDataGenerator (
    #width_shift_range=0,
    #height_shift_range=0,
    #rotation_range=0,
    #shear_range=0.01,
    #zoom_range=0.2,
    #fill_mode='constant',cval=255 # beste fill mode bei einfarbigen Hintergrund
)
# generate one image:
for batch in gen.flow(
    X_raw,
    shuffle=False,
):
    #print(batch.shape)
    plt.imshow(batch[0]/255)
    plt.show
    break
    
print(batch.shape)

In [ ]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

train_datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    brightness_range=None,
    shear_range=0,
    zoom_range=0.2,
    channel_shift_range=0.0,
    fill_mode='constant',
    cval=255,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1.0/255.0,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)




itr = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    color_mode="grayscale",
    classes=None,
    class_mode="categorical",
    batch_size=10,
    shuffle=False,
    seed=None,
    save_to_dir='Dataset_vid',
    #save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)
    
    
#target_size=(img_width, img_height),
#batch_size=10,
#class_mode='categorical')

i=0

for i in range(0,3):
    X, y = itr.next()
    

In [ ]:
from numpy import save, load
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('Dataset_vid/X_train.npy', X_train)
save('Dataset_vid/y_train.npy', y_train)
save('Dataset_vid/X_test.npy', X_test)
save('Dataset_vid/y_test.npy', y_test)